# Multipenalty Optimized Models

This notebook gives examples of using multipenalty optimized classifiers and regressors. These models work just like scikit-learn models, with the exception that the `.fit` method requires demographics.  They do not require demographics when calling `.predict`.

In [1]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge, LogisticRegression

from ai_mitigation.models import MPORegressor, MPOClassifier
from ai_mitigation.demo_utils import convert_df_to_demo_dicts
from ai_mitigation.utils import calc_cohens_d_pairwise

from example_data import gen_example_dataset

In [2]:
data = gen_example_dataset()

## Demographics Data Structures

We support two types of demographic data structures: demographic dataframes and demographic dictionaries. Both structures will work, but sometimes one is more convenient to use than the other.

A demographic dataframe has demographic columns (by convention, they must start with `demo_` to distinguish them from other columns). The example I'm showing only has a single column, but these models will optimize on multiple demographic categories.

In [3]:
X = data["X"]
X_test = data["X_test"]
y = data["y"]
y_test = data["y_test"]
demo = data["demo"]
demo_test = data["demo_test"]

In [4]:
demo.head()

,demo_Gender
0,Male
1,Female
2,Female
3,Male
4,Female


Demographic dictionaries are nested dictionaries that follow the form `{demo_category: {demo_group: mask, ...}, ...}`

In [5]:
demo_dicts = convert_df_to_demo_dicts(demo)

In [6]:
demo_dicts

{'demo_Gender': {'Female': array([False,  True,  True, ...,  True,  True, False]),
  'Male': array([ True, False, False, ..., False, False,  True])}}

# Usage: Regressor

In [7]:
model_sklearn = Ridge(solver="cholesky")
model0 = MPORegressor(beta=0)
model3 = MPORegressor(beta=3)

In [8]:
model_sklearn.fit(X, y)
model0.fit(X, y, demo)
model3.fit(X, y, demo)

When `beta` is 0, the MPORegressor is equivalent to Ridge

In [9]:
print(model_sklearn.coef_)

[0.01859943 0.02094763 0.020763   0.02098842 0.02116851 0.02096668
 0.02331898 0.023183   0.02552222 0.02429843 0.02420451]


In [10]:
print(model0.coef_)

[0.0185982  0.02094671 0.02076224 0.02098784 0.02116832 0.02096703
 0.02331984 0.02318355 0.02552329 0.02429975 0.02420621]


In [11]:
y_pred_sklearn = model_sklearn.predict(X_test)
y_pred0 = model0.predict(X_test)
y_pred3 = model3.predict(X_test)

In [12]:
print("Scikit-learn Ridge")
print("Pearson's R =", pearsonr(y_pred_sklearn, y_test)[0])
print(
    "Cohen's D (Male-Female) =",
    calc_cohens_d_pairwise(y_pred_sklearn, demo_test).d[("Male", "Female")],
)

Sklearn Ridge
Pearson's R = 0.508406517066743
Cohen's D (Male-Female) = 0.33624877337032144


In [13]:
print("Beta=0")
print("Pearson's R =", pearsonr(y_pred0, y_test)[0])
print("Cohen's D (Male-Female) =", calc_cohens_d_pairwise(y_pred0, demo_test).d[("Male", "Female")])

Beta=0
Pearson's R = 0.508407257676023
Cohen's D (Male-Female) = 0.33627054419587504


However, when beta > 0, group differences are decreased, but predictive validity is lower.

In [14]:
print("Beta=3")
print("Pearson's R =", pearsonr(y_pred3, y_test)[0])
print("Cohen's D (Male-Female) =", calc_cohens_d_pairwise(y_pred3, demo_test).d[("Male", "Female")])

Beta=3
Pearson's R = 0.49290152488821115
Cohen's D (Male-Female) = 0.2094576493017532


# Usage: Classifier

We also have a logistic-regression based classifier that incorporates a group differences term. Similarly, when beta=0, it is equivalent to regular l2-logistic regression

In [15]:
model_sklearn = LogisticRegression(solver="lbfgs")
model0 = MPOClassifier(beta=0)
model3 = MPOClassifier(beta=3)

In [16]:
model_sklearn.fit(X, y > 0)
model0.fit(X, y > 0, demo)
model3.fit(X, y > 0, demo)

In [17]:
y_pred_sklearn = model_sklearn.predict_proba(X_test)[:, 1]
y_pred0 = model0.predict_proba(X_test)[:, 1]
y_pred3 = model3.predict_proba(X_test)[:, 1]

In [18]:
print("Sklearn Logitic Regression")
print("AUC =", roc_auc_score(y_test > 0, y_pred_sklearn))
passing_rate_male = np.mean(y_pred_sklearn[demo_test.demo_Gender == "Male"] > 0.5)
passing_rate_female = np.mean(y_pred_sklearn[demo_test.demo_Gender == "Female"] > 0.5)
print("Adverse Impact Ratio (Female/Male) =", passing_rate_female / passing_rate_male)

Sklearn Logitic Regression
AUC = 0.7400576658925633
Adverse Impact Ratio (Female/Male) = 0.8034714004480716


In [19]:
print("Beta=0")
print("AUC =", roc_auc_score(y_test > 0, y_pred0))
passing_rate_male = np.mean(y_pred0[demo_test.demo_Gender == "Male"] > 0.5)
passing_rate_female = np.mean(y_pred0[demo_test.demo_Gender == "Female"] > 0.5)
print("Adverse Impact Ratio (Female/Male) =", passing_rate_female / passing_rate_male)

Beta=0
AUC = 0.7400563459293572
Adverse Impact Ratio (Female/Male) = 0.8039339631426302


In [20]:
print("Beta=3")
print("AUC =", roc_auc_score(y_test > 0, y_pred3))
passing_rate_male = np.mean(y_pred3[demo_test.demo_Gender == "Male"] > 0.5)
passing_rate_female = np.mean(y_pred3[demo_test.demo_Gender == "Female"] > 0.5)
print("Adverse Impact Ratio (Female/Male) =", passing_rate_female / passing_rate_male)

Beta=3
AUC = 0.7123054394858743
Adverse Impact Ratio (Female/Male) = 0.9480857812472467
